In [1]:
from collections import Counter
import math
from operator import itemgetter


def triplets(s: bytes):
    s = b' ' + s.strip() + b' '
    memview = memoryview(s)
    for i in range(len(s)-2):
        yield memview[i:i+3]
        

def all_triplets_from_many_lines(lines):
    for line in lines:
        yield from triplets(line)
        

def load_file(filename):
    with open(filename, 'rb') as file:
        return {line.rstrip(b'\r\n'): i for i, line in enumerate(file.readlines())}
    
    
def account_triplets(lines):
    c = Counter(all_triplets_from_many_lines(lines))
    m = max(c.values())
    for key in c:
        c[key] /= m  # Нормализуем по максимальному значению
    return c


def get_score(s: bytes):
    # return sum(c[t] for t in triplets(s)) / len(s)
    return sum(c[t] for t in triplets(s)) / math.log(len(s)+1)  # Преимущество длинным строкам

In [2]:
old_file = load_file('../stringdumps/stringdump_0_47_04.txt')
old_file.update(load_file('../stringdumps/stringdump_0_47_05.txt'))
old_file.update(load_file('../stringdumps/stringdump_0_47_03.txt'))
old_file.update(load_file('../stringdumps/stringdump_0_47_02.txt'))
old_file.update(load_file('../stringdumps/stringdump_0_47_01.txt'))
old_file.update(load_file('../stringdumps/stringdump_0_44_12.txt'))
old_file.update(load_file('../stringdumps/stringdump_steam_50_01.txt'))
old_file.update(load_file('../stringdumps/stringdump_steam_50_02.txt'))
old_file.update(load_file('../stringdumps/stringdump_steam_50_05.txt'))
old_file.update(load_file('../stringdumps/stringdump_steam_50_06.txt'))
old_file.update(load_file('../stringdumps/stringdump_steam_50_08.txt'))
old_file.update(load_file('../stringdumps/stringdump_steam_50_09.txt'))

c = account_triplets(old_file)  # Обучаем на старых файлах

In [4]:
new_file = load_file('./stringdump_50_10.txt')

In [5]:
diff = sorted(new_file.keys()-old_file.keys(), key=lambda s: get_score(s))

In [6]:
for item in diff:
    print(item, get_score(item))

b"'\xe2\x95\x92Z" 0.0
b'B\xe2\x89\xa1\xe2\x95\x95' 0.0
b'\xe2\x96\x88C' 0.0
b'\xe2\x89\xa1r\xe2\x94\x82' 0.0
b'P\xe2\x8c\xa1}' 0.0
b'nB\xe2\x89\xa1\xe2\x95\x95' 0.0
b'\x7fEq\xe2\x89\xa1\xe2\x95\x95' 0.0
b'\xce\xb1B' 0.0
b'\xe2\x96\x91V' 0.0
b'\xe2\x8c\xa1c\xe2\x89\xa1\xe2\x95\x95' 0.0
b'u\xe2\x94\x8c\xe2\x89\xa1\xe2\x95\x95' 0.0
b'u#\xe2\x89\xa1\xe2\x95\x95' 0.0
b'!EU7\xe2\x89\xa1\xe2\x95\x95' 0.0
b'iz\xe2\x89\xa1\xe2\x95\x95' 0.0
b'\xce\xb1Z' 0.0
b'<E' 0.0
b"'dG" 0.0
b"'d<" 0.0
b'\xe2\x89\xa1\xe2\x94\xacH' 0.0
b'K\xc2\xbdb' 0.0
b'1k\xc3\xb3' 0.0
b'\xe2\x95\xa8X' 0.0
b'\xc3\xaeB\xe2\x89\xa1\xe2\x95\x95' 0.0
b'h' 0.0
b'`\xe2\x96\x90q' 0.0
b'-x ' 0.0
b'\xc2\xbd\xc3\xa6e' 0.0
b'\xc3\xa1A' 0.0
b'P9' 0.0
b'u\xc3\xbc"' 0.0
b't\xe2\x94\xbcf\xe2\x89\xa1\xe2\x95\x95' 0.0
b'NB\xe2\x89\xa1\xe2\x95\x95' 0.0
b'PB\xce\xb1' 0.0
b'2G' 0.0
b'\xe2\x81\xbf\xe2\x8c\x90\xc2\xb1\xe2\x95\xa5MbP?' 0.0
b'H\xc3\xad\xe2\x94\xb4' 0.0
b'uA\xe2\x89\xa1\xe2\x95\x95' 0.0
b"'t{" 0.0
b'\xe2\x94\x94YD' 0.0
b'k\xc2\xa2\x

In [7]:
threshold = 0.01

In [10]:
with open('../stringdumps/stringdump_steam_50_10.txt', 'wb') as output:
    for line, number in sorted(new_file.items(), key=itemgetter(1)):
        if line in diff and get_score(line) < threshold: # Отсеиваем только добавившиеся строки
            continue
        
        output.write(line)
        output.write(b'\n')